# Data Pre-Processing
This notebook serves to clean the NASDAQ100 dataset to get it into a form that replicates the structure of the CHinese Stock Index datasets. 

In [ ]:
import pandas as pd
import numpy as np
import os
import joblib as joblib
import pickle

from tqdm.auto import tqdm
from datetime import datetime
from scipy.stats import zscore

# from ReMASTER.system import get_data_dir

tqdm.pandas()

The NASDAQ100 dataset was downloaded from the following website:

https://www.kaggle.com/datasets/salaheddineelkhirani/5-year-data-for-s-and-p-500-and-nasdaq-100?select=NQ_5Years_8_11_2024.csv 

In [ ]:
# locally
# ----------------------------------------------------------------
# file_path = '../data/NQ_5Years_8_11_2024.csv'

# # Initialize an empty DataFrame for the combined data
# # data = pd.DataFrame()
# data = pd.read_csv(file_path)

# # Display the first few rows of the dataframe to inspect its structure
# data.head()

# on google colaboratory
# ----------------------------------------------------------------
from google.colab import drive
import os

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Set the path to your data folder
data_path = '/content/drive/My Drive/ECE 570/data'

# data from https://www.kaggle.com/datasets/salaheddineelkhirani/5-year-data-for-s-and-p-500-and-nasdaq-100?select=NQ_5Years_8_11_2024.csv

# Step 3: List files in the folder (to verify)
print("Files in data folder:")
for filename in os.listdir(data_path):
    print(filename)

# Step 4: Load the data
file_path = '/content/drive/My Drive/ECE 570/data/NQ_5Years_8_11_2024.csv'

# Initialize an empty DataFrame for the combined data
# data = pd.DataFrame()
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to inspect its structure
data.head()

In [ ]:
# 1. Convert 'Date' to datetime format
data['Date'] = pd.to_datetime(data['Time'])

# Set 'Date' as the index for time-based operations
data.set_index('Date', inplace=True)

# Verify the structure (no need to drop 'Time', as it's part of the index now)
data.head()

## updated method to ensure this set fits the csi300 form with alpha158 form etc.

Split data into train, valid, and test sets by datetime. Earlier years are in training, recent years in test. 

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import linregress

def calculate_csi300_features(df, instrument_id='NQ100'):
    """Calculate features and reshape data into CSI300 format."""
    # Make a copy of the dataframe
    df = df.copy()

    # Create features dictionary
    features = {}

    # Price Change Ratios
    features['(close-open)/open'] = (df['Close'] - df['Open']) / df['Open']
    features['(high-low)/open'] = (df['High'] - df['Low']) / df['Open']
    features['(2*close-high-low)/open'] = (2 * df['Close'] - df['High'] - df['Low']) / df['Open']
    print("Price Change Ratios done")

    # Moving Averages and Standard Deviations (Volatility)
    features['Mean(close,5)/close'] = df['Close'].rolling(window=5).mean() / df['Close']
    features['Mean(close,10)/close'] = df['Close'].rolling(window=10).mean() / df['Close']
    features['Mean(close,20)/close'] = df['Close'].rolling(window=20).mean() / df['Close']
    features['Std(close,5)/close'] = df['Close'].rolling(window=5).std() / df['Close']
    features['Std(close,10)/close'] = df['Close'].rolling(window=10).std() / df['Close']
    features['Std(close,20)/close'] = df['Close'].rolling(window=20).std() / df['Close']
    print("Moving Averages and Standard Deviations done")

    # Relative Strength and Trends (Slope calculation)
    for window in [5, 10, 20]:
        features[f'Slope(close,{window})/close'] = df['Close'].rolling(window).apply(
            lambda x: linregress(range(len(x)), x).slope / x[-1] if len(x) == window else np.nan
        )
    print("Relative Strength and Trends done")

    # Volume-Weighted Measures
    features['VWAP/close'] = (df['Volume'] * df['Close']).cumsum() / df['Volume'].cumsum() / df['Close']
    features['Mean(volume,5)/(volume+1e-12)'] = df['Volume'].rolling(window=5).mean() / (df['Volume'] + 1e-12)
    features['Std(volume,5)/(volume+1e-12)'] = df['Volume'].rolling(window=5).std() / (df['Volume'] + 1e-12)
    print("Volume-Weighted Measures done")

    # Historical Close Ratios
    features['Ref(close,5)/close'] = df['Close'].shift(5) / df['Close']
    features['Ref(close,10)/close'] = df['Close'].shift(10) / df['Close']
    features['Ref(close,20)/close'] = df['Close'].shift(20) / df['Close']
    print("Historical Close Ratios done")

    # Correlation Measures
    features['Corr(close,log(volume+1),5)'] = df['Close'].rolling(window=5).corr(np.log(df['Volume'] + 1))
    features['Corr(close/Ref(close,1),log(volume/Ref(volume,1)+1),5)'] = (
        (df['Close'] / df['Close'].shift(1)).rolling(window=5)
        .corr(np.log(df['Volume'] / df['Volume'].shift(1) + 1))
    )
    print("Correlation Measures done")

    # Price Extremes and Ratios
    features['Max(high,5)/close'] = df['High'].rolling(window=5).max() / df['Close']
    features['Min(low,5)/close'] = df['Low'].rolling(window=5).min() / df['Close']
    print("Price Extremes and Ratios done")

    # Create DataFrame from features with the same index as input data
    feature_df = pd.DataFrame(features, index=df.index)

    # Add datetime (using the index date) and instrument columns
    feature_df['datetime'] = feature_df.index.date
    feature_df['instrument'] = instrument_id

    # Reorder columns to put datetime and instrument first
    cols = ['datetime', 'instrument'] + [col for col in feature_df.columns if col not in ['datetime', 'instrument']]
    feature_df = feature_df[cols]

    return feature_df

def process_single_instrument(data, instrument_id, date_ranges):
    """Process data for a single instrument."""
    # Calculate features
    processed_df = calculate_csi300_features(data, instrument_id)

    # Set datetime and instrument as index
    processed_df.set_index(['datetime', 'instrument'], inplace=True)

    # Split data based on date ranges
    train_data = processed_df[processed_df.index.get_level_values(0) <= date_ranges['train_end'].date()]
    valid_data = processed_df[
        (processed_df.index.get_level_values(0) >= date_ranges['valid_start'].date()) &
        (processed_df.index.get_level_values(0) <= date_ranges['valid_end'].date())
    ]
    test_data = processed_df[
        (processed_df.index.get_level_values(0) >= date_ranges['test_start'].date()) &
        (processed_df.index.get_level_values(0) <= date_ranges['test_end'].date())
    ]

    return train_data, valid_data, test_data

# Debug: Print input data structure
print("Input data structure:")
print(data.head())
print("\nInput data info:")
print(data.info())

# Define date ranges
date_ranges = {
    'train_end': datetime(2022, 3, 31),
    'valid_start': datetime(2022, 4, 1),
    'valid_end': datetime(2022, 6, 30),
    'test_start': datetime(2022, 7, 1),
    'test_end': datetime(2024, 12, 31)
}

# Process the data for NQ100
instrument_id = 'NQ100'
train_data, valid_data, test_data = process_single_instrument(data, instrument_id, date_ranges)

# Save the processed datasets
train_data.to_csv('train_data_2020_2022_NQ100.csv')
valid_data.to_csv('valid_data_2022_NQ100.csv')
test_data.to_csv('test_data_2022_2024_NQ100.csv')

# Print sample output to verify format
print("\nSample of training data:")
print(train_data.head())

# Print shape information
print("\nDataset shapes:")
print(f"Train data: {train_data.shape}")
print(f"Validation data: {valid_data.shape}")
print(f"Test data: {test_data.shape}")

# convert to .pkl files

In [ ]:
# 6. Save each set as .pkl files
# locally
train_path = '../data/NQ100/NQ100_dl_train.pkl'
valid_path = '../data/NQ100/NQ100_dl_valid.pkl'
test_path = '../data/NQ100/NQ100_dl_test.pkl'

# google drive for colab
train_path = '/content/drive/My Drive/ECE 570/data/NQ100/NQ100_dl_train.pkl'
valid_path = '/content/drive/My Drive/ECE 570/data/NQ100/NQ100_dl_valid.pkl'
test_path = '/content/drive/My Drive/ECE 570/data/NQ100/NQ100_dl_test.pkl'

# Save train data
with open(train_path, 'wb') as f:
    pickle.dump(train_data, f)

# Save validation data
with open(valid_path, 'wb') as f:
    pickle.dump(valid_data, f)

# Save test data
with open(test_path, 'wb') as f:
    pickle.dump(test_data, f)

# Return the paths (optional)
train_path, valid_path, test_path

## ensure pkl files have content 

In [ ]:
# Function to load and test if data exists in each .pkl file
def test_data_in_pkl(file_path):
    try:
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        # Check if the DataFrame is not empty
        if isinstance(data, pd.DataFrame) and not data.empty:
            print(f"Data exists in {file_path} and has {len(data)} rows and {len(data.columns)} columns.")
        else:
            print(f"File {file_path} is either empty or not a DataFrame.")
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

# Test each .pkl file
test_data_in_pkl(train_path)
test_data_in_pkl(valid_path)
test_data_in_pkl(test_path)



# -